# Magic SVM Weighted + Area Under the Curve

In [24]:
import pandas as pd
from sklearn import svm
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

In [25]:
url = "https://raw.githubusercontent.com/maxxxxc/SIR-Summer-2023/main/dataset/magic04.data"
column_names = ["X1", "X2", "X3", "X4", "X5", "X6", "X7", "X8", "X9", "X10", "Y"]
df = pd.read_csv(url, header=None, names=column_names)

X = df.drop("Y", axis=1)
y = df["Y"]
print(y.value_counts())
y = y.replace({'g': 1, 'h': 0})
print(y.value_counts())

g    12332
h     6688
Name: Y, dtype: int64
1    12332
0     6688
Name: Y, dtype: int64


In [26]:
def iterate_process(X, y):
    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    
    # Define the number of batches
    num_batches = 11
    
    # Randomly shuffle the data indices
    #indices = np.random.permutation(len(X))
    
    #change 7/12
    indices = np.random.permutation(len(X_train))
    
    # Calculate the batch size
    batch_size = len(X_train) // num_batches
    
    # Make predictions on the test set using majority voting
    preds_voting = np.zeros(len(y_test))
    # Make predictions on the test set using average of logit
    preds_logit = np.zeros(len(y_test))
    #Make predictions on the test set using average of probs
    preds_prob = np.zeros(len(y_test))
    
    preds_voting_weighted = np.zeros(len(y_test))
    preds_logit_weighted = np.zeros(len(y_test))
    preds_prob_weighted = np.zeros(len(y_test))
    
    total_cverr = 0
    
    # Split the training data into batches, fit a logistic regression model on each batch
    for i in range(num_batches):
        # Calculate the starting and ending indices for the current batch
        start_index = i * batch_size
        end_index = (i + 1) * batch_size
        
        # Create a logistic regression model
        model = LogisticRegression(max_iter=500, penalty = None)
        
        # Select the current batch for training
        #X_batch = X_train[start_index:end_index]
        #y_batch = y_train[start_index:end_index]
        
        #change 7/12
        X_batch = X_train.iloc[indices[start_index:end_index]]
        y_batch = y_train.iloc[indices[start_index:end_index]]
        
        scaler = StandardScaler()
        X_batch_scaled = scaler.fit_transform(X_batch)
        X_test_scaled = scaler.transform(X_test)
        
        # Fit the model on the current batch
        model.fit(X_batch_scaled, y_batch)
        current_cverr = cross_val_score(model, X_batch_scaled, y_batch, cv = 5, scoring = 'accuracy').mean()
        total_cverr += current_cverr
               
        y_pred = model.predict(X_test_scaled)
        # Accumulate the predictions using majority voting
        preds_voting += (y_pred == 1)
        preds_voting_weighted += (y_pred == 1) * current_cverr
    
        # Accumulate the predictions using majority voting
        y_pred = model.decision_function(X_test_scaled)
        preds_logit += y_pred
        preds_logit_weighted += y_pred * current_cverr
        
        #Accumulate the probs
        y_pred = model.predict_proba(X_test_scaled)
        preds_prob += y_pred[:,1]
        preds_prob_weighted += y_pred[:,1] * current_cverr
    
    accuracy = np.zeros(7)
    auc_accuracy = np.zeros(7)
    
    preds_voting_weighted = preds_voting_weighted / total_cverr * num_batches
    preds_logit_weighted = preds_logit_weighted / total_cverr * num_batches
    preds_prob_weighted = preds_prob_weighted / total_cverr * num_batches
    
    # Majority voting (selecting the most frequent prediction for each sample)
    final_predictions = np.where(preds_voting > num_batches / 2, 1, 0)
    accuracy[0] = accuracy_score(y_test, final_predictions)
    auc_accuracy[0] = roc_auc_score(y_test, preds_voting)
    
    final_predictions = np.where(preds_voting_weighted > num_batches / 2, 1, 0)
    accuracy[1] = accuracy_score(y_test, final_predictions)
    auc_accuracy[1] = roc_auc_score(y_test, preds_voting_weighted)
    
    # Average of logit
    final_predictions = np.where(preds_logit > 0, 1, 0)
    accuracy[2] = accuracy_score(y_test, final_predictions)
    auc_accuracy[2] = roc_auc_score(y_test, preds_logit)
    
    final_predictions = np.where(preds_logit_weighted > 0, 1, 0)
    accuracy[3] = accuracy_score(y_test, final_predictions)
    auc_accuracy[3] = roc_auc_score(y_test, preds_logit_weighted)
    
    #Average of probs
    final_predictions = np.where(preds_prob / num_batches > 0.5, 1, 0)
    accuracy[4] = accuracy_score(y_test, final_predictions)
    auc_accuracy[4] = roc_auc_score(y_test, preds_prob)
    
    final_predictions = np.where(preds_prob_weighted / num_batches > 0.5, 1, 0)
    accuracy[5] = accuracy_score(y_test, final_predictions)
    auc_accuracy[5] = roc_auc_score(y_test, preds_prob_weighted)
    
    # Train a model on all 11 batches of training data
    model = LogisticRegression(max_iter=500, penalty = None)
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    accuracy[6] = accuracy_score(y_test, y_pred)
    y_pred = model.decision_function(X_test_scaled)
    auc_accuracy[6] = roc_auc_score(y_test, y_pred)
    
    return accuracy, auc_accuracy

In [7]:
# Number of times to repeat the process
num_repeats = 50

# Initialize an empty matrix (10-by-4) to store accuracies
accuracies = np.zeros((num_repeats, 7))
auc_accuracies = np.zeros((num_repeats, 7))

seed = 42
# Repeat the process and store accuracies
for i in range(num_repeats):
    np.random.seed(seed)
    accuracies[i], auc_accuracies[i] = iterate_process(X, y)
    seed += 2
    

# Print the accuracies
print("Accuracies:", accuracies)
print("AUC:", auc_accuracies)

Accuracies: [[0.79022082 0.79022082 0.78956362 0.78956362 0.78943218 0.78943218
  0.78890641]
 [0.78470032 0.78470032 0.78640904 0.7862776  0.78640904 0.78614616
  0.78588328]
 [0.7904837  0.7904837  0.78956362 0.78969506 0.78969506 0.78969506
  0.78930074]
 [0.79035226 0.79035226 0.79022082 0.79022082 0.79022082 0.79022082
  0.7904837 ]
 [0.79837014 0.79837014 0.79994742 0.80007886 0.8002103  0.80007886
  0.79994742]
 [0.78233438 0.78233438 0.78312303 0.78325447 0.78312303 0.78325447
  0.78299159]
 [0.79179811 0.79179811 0.79232387 0.79232387 0.79232387 0.79232387
  0.79179811]
 [0.78759201 0.78759201 0.78693481 0.78706625 0.78706625 0.78706625
  0.78746057]
 [0.79258675 0.79258675 0.79087802 0.79100946 0.79100946 0.79100946
  0.7904837 ]
 [0.79337539 0.79337539 0.79298107 0.79311251 0.79298107 0.79311251
  0.79284963]
 [0.7898265  0.7898265  0.78930074 0.78943218 0.78956362 0.78943218
  0.78969506]
 [0.79350683 0.79350683 0.79324395 0.79324395 0.79337539 0.79337539
  0.79376972]
 [0.

In [8]:
np.mean(accuracies, axis = 0) 

array([0.79118034, 0.79118034, 0.79110147, 0.79111199, 0.79112776,
       0.79113565, 0.79099369])

In [9]:
np.std(accuracies, axis = 0)

array([0.00375196, 0.00375196, 0.00368917, 0.00367632, 0.00369039,
       0.0037056 , 0.00380476])

In [10]:
np.mean(auc_accuracies, axis = 0)

array([0.77764808, 0.77768348, 0.83859946, 0.83859245, 0.83879207,
       0.83878592, 0.8386754 ])

In [11]:
np.std(auc_accuracies, axis = 0)

array([0.00460755, 0.00461683, 0.00365979, 0.00365989, 0.00367112,
       0.00367049, 0.0036709 ])

# Wireless Indoor Localization 

In [12]:
url = "https://raw.githubusercontent.com/maxxxxc/SIR-Summer-2023/main/dataset/wifi_localization.txt"
df = pd.read_csv(url, sep = '\t', header = None)

X = df.drop(7, axis=1)
y = df[7]
print(y.value_counts())
y = y.replace({2 : 0, 1 : 0, 3: 1, 4 : 1})
y.value_counts()

1    500
2    500
3    500
4    500
Name: 7, dtype: int64


0    1000
1    1000
Name: 7, dtype: int64

In [13]:
# Number of times to repeat the process
num_repeats = 50

# Initialize an empty matrix (10-by-4) to store accuracies
accuracies = np.zeros((num_repeats, 7))
auc_accuracies = np.zeros((num_repeats, 7))

seed = 42
# Repeat the process and store accuracies
for i in range(num_repeats):
    np.random.seed(seed)
    accuracies[i], auc_accuracies[i] = iterate_process(X, y)
    seed += 2
    

# Print the accuracies
print("Accuracies:", accuracies)
print("AUC:", auc_accuracies)

Accuracies: [[0.92875 0.92875 0.9225  0.9225  0.93    0.93    0.93125]
 [0.91375 0.91375 0.9175  0.9175  0.9225  0.9225  0.9075 ]
 [0.93    0.93    0.93125 0.93    0.93125 0.93125 0.93375]
 [0.91375 0.91375 0.9175  0.9175  0.92    0.92    0.91875]
 [0.91375 0.91375 0.905   0.905   0.91875 0.9175  0.915  ]
 [0.92125 0.92125 0.9025  0.90125 0.91875 0.9175  0.91625]
 [0.92625 0.92625 0.925   0.925   0.92625 0.92625 0.9225 ]
 [0.915   0.915   0.92    0.92    0.92125 0.92125 0.92   ]
 [0.93625 0.93625 0.90125 0.90125 0.9375  0.9375  0.93875]
 [0.92625 0.92625 0.91375 0.91375 0.92625 0.92625 0.9225 ]
 [0.9225  0.9225  0.92375 0.92375 0.92375 0.92375 0.925  ]
 [0.91625 0.91625 0.915   0.915   0.91375 0.91625 0.9175 ]
 [0.92    0.92    0.90125 0.9025  0.92125 0.92    0.92375]
 [0.9375  0.9375  0.92375 0.92375 0.93875 0.93875 0.9325 ]
 [0.92875 0.92875 0.92125 0.92125 0.93125 0.93125 0.9275 ]
 [0.905   0.905   0.9125  0.9125  0.91375 0.9125  0.91625]
 [0.92375 0.92375 0.9125  0.91    0.91875 0.

In [14]:
np.mean(accuracies, axis = 0) 

array([0.92185 , 0.92185 , 0.913125, 0.913   , 0.922675, 0.9225  ,
       0.923225])

In [15]:
np.std(accuracies, axis = 0)

array([0.00779519, 0.00779519, 0.01305816, 0.01310296, 0.00691922,
       0.00694622, 0.00683781])

In [16]:
np.mean(auc_accuracies, axis = 0)

array([0.96889311, 0.96888597, 0.97603259, 0.97599594, 0.98022505,
       0.98021392, 0.98056168])

In [17]:
np.std(auc_accuracies, axis = 0)

array([0.00488809, 0.00491924, 0.00664074, 0.0066586 , 0.00259353,
       0.002599  , 0.00236217])

In [22]:
mean = 0.92185
true = 0
false = 0

for i in range(len(accuracies)):
    if (abs(accuracies[i][0] - mean) <= 2*0.007795190825117758):
        true += 1
    else:
        false += 1

In [23]:
print(true)
print(false)

48
2


# Turkiye Student Evaluation

In [44]:
url = "https://raw.githubusercontent.com/maxxxxc/SIR-Summer-2023/main/dataset/turkiye-student-evaluation_generic.csv"
df = pd.read_csv(url)

X = df.drop("difficulty", axis=1)
y = df["difficulty"]

X = X.drop("instr", axis=1)
X = X.drop("class", axis=1)
X = X.drop("nb.repeat", axis=1)
X = X.drop("attendance", axis=1)

print(y.value_counts())
y = y.replace({2 : 0, 1 : 0, 3: 1, 4 : 1, 5 : 1})
y.value_counts()

3    1774
1    1620
4    1225
5     652
2     549
Name: difficulty, dtype: int64


1    3651
0    2169
Name: difficulty, dtype: int64

In [45]:
# Number of times to repeat the process
num_repeats = 50

# Initialize an empty matrix (10-by-4) to store accuracies
accuracies = np.zeros((num_repeats, 7))
auc_accuracies = np.zeros((num_repeats, 7))

seed = 42
# Repeat the process and store accuracies
for i in range(num_repeats):
    np.random.seed(seed)
    accuracies[i], auc_accuracies[i] = iterate_process(X, y)
    seed += 2
    

# Print the accuracies
print("Accuracies:", accuracies)
print("AUC:", auc_accuracies)

Accuracies: [[0.64905498 0.64905498 0.65506873 0.65463918 0.65678694 0.65635739
  0.65592784]
 [0.64175258 0.64175258 0.63960481 0.63960481 0.63917526 0.6387457
  0.64046392]
 [0.64218213 0.64218213 0.64862543 0.64948454 0.64690722 0.64776632
  0.6516323 ]
 [0.6516323  0.6516323  0.65034364 0.65034364 0.65335052 0.65206186
  0.65249141]
 [0.64475945 0.64475945 0.64304124 0.64475945 0.64733677 0.64647766
  0.6443299 ]
 [0.63960481 0.63960481 0.6387457  0.63745704 0.63831615 0.6378866
  0.64604811]
 [0.64690722 0.64690722 0.64819588 0.64862543 0.64776632 0.64905498
  0.64905498]
 [0.63659794 0.63659794 0.63831615 0.63917526 0.6387457  0.6387457
  0.64304124]
 [0.63101375 0.63101375 0.62800687 0.62886598 0.62929553 0.62886598
  0.63702749]
 [0.63015464 0.63015464 0.63230241 0.63187285 0.62886598 0.63058419
  0.6378866 ]
 [0.64905498 0.64905498 0.64347079 0.645189   0.64647766 0.645189
  0.64690722]
 [0.65721649 0.65721649 0.65678694 0.65549828 0.65721649 0.65721649
  0.65979381]
 [0.63445

In [46]:
np.mean(accuracies, axis = 0) 

array([0.63900344, 0.63900344, 0.64071306, 0.64094502, 0.64089347,
       0.64069588, 0.64311856])

In [47]:
np.std(accuracies, axis = 0)

array([0.00840389, 0.00840389, 0.00834886, 0.00834733, 0.00861252,
       0.00838992, 0.00862737])

In [48]:
np.mean(auc_accuracies, axis = 0)

array([0.56219138, 0.5624432 , 0.60241535, 0.60216581, 0.59881968,
       0.59862326, 0.60124213])

In [50]:
np.std(auc_accuracies, axis = 0)

array([0.00908551, 0.00915442, 0.00855315, 0.00858387, 0.00949924,
       0.0094652 , 0.00980886])

# Tree Wilt

In [51]:
url = "https://raw.githubusercontent.com/maxxxxc/SIR-Summer-2023/main/dataset/Wilt%20Dataset.csv"
df = pd.read_csv(url, header = None)

X = df.drop(0, axis=1)
y = df[0]

y.value_counts()

0    4578
1     261
Name: 0, dtype: int64

In [52]:
# Number of times to repeat the process
num_repeats = 50

# Initialize an empty matrix (10-by-4) to store accuracies
accuracies = np.zeros((num_repeats, 7))
auc_accuracies = np.zeros((num_repeats, 7))

seed = 42
# Repeat the process and store accuracies
for i in range(num_repeats):
    np.random.seed(seed)
    accuracies[i], auc_accuracies[i] = iterate_process(X, y)
    seed += 2
    

# Print the accuracies
print("Accuracies:", accuracies)
print("AUC:", auc_accuracies)

Accuracies: [[0.96745868 0.96745868 0.9643595  0.9643595  0.96694215 0.96694215
  0.96590909]
 [0.97159091 0.97159091 0.97055785 0.97055785 0.97055785 0.97055785
  0.97004132]
 [0.96952479 0.96952479 0.9643595  0.9643595  0.96849174 0.96849174
  0.96900826]
 [0.96745868 0.96745868 0.96745868 0.96745868 0.96745868 0.96745868
  0.96539256]
 [0.97004132 0.97004132 0.97159091 0.97159091 0.97210744 0.97210744
  0.97004132]
 [0.96694215 0.96694215 0.96745868 0.96745868 0.97004132 0.97004132
  0.96539256]
 [0.96745868 0.96745868 0.96694215 0.96694215 0.96797521 0.96797521
  0.96280992]
 [0.96539256 0.96539256 0.96539256 0.96539256 0.96539256 0.96539256
  0.9643595 ]
 [0.97159091 0.97159091 0.97210744 0.97210744 0.97210744 0.97210744
  0.96849174]
 [0.96849174 0.96849174 0.96900826 0.96900826 0.96900826 0.96900826
  0.96849174]
 [0.97210744 0.97210744 0.97107438 0.97107438 0.9731405  0.9731405
  0.97107438]
 [0.9731405  0.9731405  0.97159091 0.97210744 0.97210744 0.97210744
  0.97159091]
 [0.9

In [53]:
np.mean(accuracies, axis = 0) 

array([0.9694938 , 0.9694938 , 0.96815083, 0.9681095 , 0.9702686 ,
       0.9702686 , 0.96779959])

In [54]:
np.std(accuracies, axis = 0)

array([0.00313752, 0.00313752, 0.00390191, 0.00395246, 0.00315406,
       0.00317766, 0.00320016])

In [55]:
np.mean(auc_accuracies, axis = 0)

array([0.91075276, 0.91083874, 0.97744843, 0.97744293, 0.97772305,
       0.97772621, 0.97678313])

In [56]:
np.std(auc_accuracies, axis = 0)

array([0.02270243, 0.02277645, 0.00525803, 0.00526137, 0.00510687,
       0.00510549, 0.00537341])

# Spambase

In [33]:
url = "https://raw.githubusercontent.com/maxxxxc/SIR-Summer-2023/main/dataset/spambase.data"
df = pd.read_csv(url, header = None)

X = df.drop(57, axis=1)
y = df[57]

y.value_counts()

0    2788
1    1813
Name: 57, dtype: int64

In [34]:
# Number of times to repeat the process
num_repeats = 50

# Initialize an empty matrix (10-by-4) to store accuracies
accuracies = np.zeros((num_repeats, 7))
auc_accuracies = np.zeros((num_repeats, 7))

seed = 42
# Repeat the process and store accuracies
for i in range(num_repeats):
    np.random.seed(seed)
    accuracies[i], auc_accuracies[i] = iterate_process(X, y)
    seed += 2
    

# Print the accuracies
print("Accuracies:", accuracies)
print("AUC:", auc_accuracies)

C:\Users\panda\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\panda\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

C:\Users\panda\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\panda\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

C:\Users\panda\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\panda\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

C:\Users\panda\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\panda\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

C:\Users\panda\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\panda\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

...

C:\Users\panda\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\panda\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

C:\Users\panda\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\panda\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

C:\Users\panda\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\panda\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

Accuracies: [[0.92667029 0.92667029 0.86094514 0.86148832 0.92667029 0.92504074
  0.92123846]
 [0.92775665 0.92775665 0.87778381 0.87778381 0.92775665 0.92884302
  0.93101575]
 [0.92178164 0.92178164 0.88973384 0.88973384 0.92178164 0.92178164
  0.91906573]
 [0.92123846 0.92123846 0.85116784 0.85171103 0.92123846 0.92123846
  0.92015209]
 [0.93155894 0.93155894 0.87072243 0.87126562 0.93155894 0.92992939
  0.9293862 ]
 [0.92558392 0.92558392 0.87724063 0.87669745 0.92558392 0.92558392
  0.92667029]
 [0.90983161 0.90983161 0.87235198 0.87072243 0.9103748  0.9103748
  0.92449756]
 [0.92829984 0.92829984 0.90983161 0.9103748  0.92775665 0.92775665
  0.92884302]
 [0.92721347 0.92721347 0.88919066 0.88919066 0.92721347 0.92775665
  0.91472026]
 [0.9293862  0.9293862  0.87995655 0.87887018 0.92829984 0.92829984
  0.91634981]
 [0.92232482 0.92232482 0.88593156 0.88484519 0.92232482 0.92232482
  0.92123846]
 [0.91689299 0.91689299 0.8332428  0.83378599 0.91689299 0.91689299
  0.91526344]
 [0.9

C:\Users\panda\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [35]:
np.mean(accuracies, axis = 0) 

array([0.92355242, 0.92355242, 0.87765345, 0.87760999, 0.92355242,
       0.92355242, 0.92410646])

In [36]:
np.std(accuracies, axis = 0)

array([0.00581872, 0.00581872, 0.02172503, 0.02186174, 0.00568223,
       0.00561538, 0.00492477])

In [37]:
np.mean(auc_accuracies, axis = 0)

array([0.97163392, 0.97170525, 0.93128172, 0.93130658, 0.97298443,
       0.97286114, 0.9708845 ])

In [38]:
np.std(auc_accuracies, axis = 0)

array([0.00360492, 0.00367017, 0.01790402, 0.01791961, 0.00330096,
       0.00337995, 0.00272569])

# default of credit card clients

In [39]:
url = "https://raw.githubusercontent.com/maxxxxc/SIR-Summer-2023/main/dataset/default%20of%20credit%20card%20clients.csv"
df = pd.read_csv(url)

X = df.drop("Y", axis=1)
y = df["Y"]

y.value_counts()

0    23364
1     6636
Name: Y, dtype: int64

In [40]:
# Number of times to repeat the process
num_repeats = 50

# Initialize an empty matrix (10-by-4) to store accuracies
accuracies = np.zeros((num_repeats, 7))
auc_accuracies = np.zeros((num_repeats, 7))

seed = 42
# Repeat the process and store accuracies
for i in range(num_repeats):
    np.random.seed(seed)
    accuracies[i], auc_accuracies[i] = iterate_process(X, y)
    seed += 2
    

# Print the accuracies
print("Accuracies:", accuracies)
print("AUC:", auc_accuracies)

Accuracies: [[0.80791667 0.80791667 0.80766667 0.8075     0.80758333 0.8075
  0.80825   ]
 [0.80841667 0.80841667 0.80858333 0.80858333 0.80883333 0.80866667
  0.80941667]
 [0.80925    0.80925    0.81108333 0.81108333 0.811      0.81108333
  0.8115    ]
 [0.81225    0.81225    0.81208333 0.81208333 0.81208333 0.81208333
  0.81275   ]
 [0.81608333 0.81608333 0.816      0.816      0.81591667 0.816
  0.81641667]
 [0.81166667 0.81166667 0.81191667 0.81216667 0.81175    0.81191667
  0.81241667]
 [0.81083333 0.81083333 0.80891667 0.80891667 0.80875    0.80883333
  0.81025   ]
 [0.81116667 0.81116667 0.81133333 0.81133333 0.81141667 0.81141667
  0.812     ]
 [0.81275    0.81275    0.81158333 0.81158333 0.81158333 0.81158333
  0.812     ]
 [0.80941667 0.80941667 0.80891667 0.809      0.80883333 0.80891667
  0.80858333]
 [0.81091667 0.81091667 0.811      0.811      0.81083333 0.811
  0.8105    ]
 [0.81033333 0.81033333 0.80941667 0.8095     0.80958333 0.80941667
  0.81091667]
 [0.80641667 0.806

In [41]:
np.mean(accuracies, axis = 0) 

array([0.81004167, 0.81004167, 0.80989167, 0.80988667, 0.80988333,
       0.80988333, 0.81037167])

In [42]:
np.std(accuracies, axis = 0)

array([0.00294554, 0.00294554, 0.00279515, 0.00281137, 0.00278976,
       0.00281302, 0.00280289])

In [43]:
np.mean(auc_accuracies, axis = 0)

array([0.66914422, 0.66916894, 0.72516297, 0.72515241, 0.72633702,
       0.7263249 , 0.72237257])

In [44]:
np.std(auc_accuracies, axis = 0)

array([0.00347597, 0.00347445, 0.00386901, 0.00386964, 0.00392266,
       0.00392418, 0.00384066])

# APS Failure at Scania Trucks

In [69]:
url = "https://raw.githubusercontent.com/maxxxxc/SIR-Summer-2023/main/dataset/aps_failure_test_set.csv"
# Read the training dataset from CSV
train_df = pd.read_csv(url, na_values = 'na')

# Read the test dataset from CSV
test_df = pd.read_csv(url, na_values = 'na')

# Add a 'Label' column to the test dataset and fill it with NaN values
#test_df['Label'] = float('nan')

# Concatenate the training and test datasets
df = pd.concat([train_df, test_df], ignore_index=True)

# Save the combined dataset to a new CSV file
df.to_csv('combined.csv', index=False)

missing_values_count = df.isna().sum()

X = df.drop("class", axis=1)
y = df["class"]

threshold = len(X) * 0.5
X = X.dropna(thresh = threshold, axis = 1)

X = X.fillna(X.mean())

y = y.replace({"neg" : 0, "pos" : 1})
y.value_counts()

0    31250
1      750
Name: class, dtype: int64

In [70]:
# Number of times to repeat the process
num_repeats = 50

# Initialize an empty matrix (10-by-4) to store accuracies
accuracies = np.zeros((num_repeats, 7))
auc_accuracies = np.zeros((num_repeats, 7))

seed = 42
# Repeat the process and store accuracies
for i in range(num_repeats):
    np.random.seed(seed)
    accuracies[i], auc_accuracies[i] = iterate_process(X, y)
    seed += 2
    

# Print the accuracies
print("Accuracies:", accuracies)
print("AUC:", auc_accuracies)

C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracies: [[0.98921875 0.98921875 0.98625    0.98625    0.98921875 0.98921875
  0.99023438]
 [0.98929687 0.98929687 0.98679688 0.98679688 0.98929687 0.98929687
  0.99125   ]
 [0.9890625  0.9890625  0.98773438 0.98773438 0.9890625  0.9890625
  0.99039062]
 [0.9896875  0.9896875  0.98882812 0.98882812 0.9896875  0.9896875
  0.99148437]
 [0.98828125 0.98828125 0.985625   0.985625   0.98851563 0.98851563
  0.98953125]
 [0.98820312 0.98820312 0.98625    0.98625    0.988125   0.988125
  0.9896875 ]
 [0.98984375 0.98984375 0.98664063 0.98664063 0.98984375 0.98984375
  0.99171875]
 [0.9875     0.9875     0.98578125 0.98578125 0.98757812 0.98765625
  0.99148437]
 [0.98890625 0.98890625 0.98492187 0.98492187 0.98890625 0.98890625
  0.99070313]
 [0.98953125 0.98953125 0.98625    0.98625    0.98953125 0.98953125
  0.99179688]
 [0.98890625 0.98890625 0.98726563 0.98726563 0.98882812 0.98882812
  0.99023438]
 [0.98945313 0.98945313 0.98789063 0.98789063 0.98945313 0.98945313
  0.99210937]
 [0.9888

C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [71]:
np.mean(accuracies, axis = 0) 

array([0.98873594, 0.98873594, 0.98654062, 0.98655469, 0.9887625 ,
       0.9887625 , 0.99050937])

In [72]:
np.std(accuracies, axis = 0)

array([0.00088866, 0.00088866, 0.00129367, 0.00127344, 0.00087184,
       0.00086987, 0.00088364])

In [73]:
np.mean(auc_accuracies, axis = 0)

array([0.98356597, 0.98361155, 0.85042364, 0.85051619, 0.99107025,
       0.99107196, 0.96107475])

In [74]:
np.std(auc_accuracies, axis = 0)

array([0.00652687, 0.00652271, 0.04930128, 0.04930909, 0.00391692,
       0.00389728, 0.01262707])

# Epileptic Seizure Recognition

In [75]:
url = "https://raw.githubusercontent.com/maxxxxc/SIR-Summer-2023/main/dataset/Epileptic%20Seizure%20Recognition.csv"
df = pd.read_csv(url)

X = df.drop("y", axis=1)
y = df["y"]

X = X.drop("Unnamed", axis=1)

y = y.replace({4 : 0, 3 : 0, 2 : 0, 5 : 0})
y.value_counts()

0    9200
1    2300
Name: y, dtype: int64

In [76]:
# Number of times to repeat the process
num_repeats = 50

# Initialize an empty matrix (10-by-4) to store accuracies
accuracies = np.zeros((num_repeats, 7))
auc_accuracies = np.zeros((num_repeats, 7))

seed = 42
# Repeat the process and store accuracies
for i in range(num_repeats):
    np.random.seed(seed)
    accuracies[i], auc_accuracies[i] = iterate_process(X, y)
    seed += 2
    

# Print the accuracies
print("Accuracies:", accuracies)
print("AUC:", auc_accuracies)

C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

...

C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

Accuracies: [[0.83043478 0.83043478 0.82782609 0.82717391 0.83021739 0.83
  0.81934783]
 [0.83347826 0.83347826 0.82152174 0.82108696 0.83369565 0.83391304
  0.81956522]
 [0.8423913  0.8423913  0.84956522 0.84956522 0.84217391 0.84173913
  0.82847826]
 [0.84673913 0.84673913 0.84369565 0.84391304 0.8473913  0.84717391
  0.82913043]
 [0.84217391 0.84217391 0.84021739 0.84043478 0.8423913  0.8426087
  0.82565217]
 [0.83630435 0.83630435 0.82891304 0.82869565 0.83652174 0.83673913
  0.81565217]
 [0.83630435 0.83630435 0.83478261 0.83369565 0.83586957 0.83565217
  0.825     ]
 [0.82695652 0.82695652 0.82021739 0.81978261 0.82695652 0.82673913
  0.81913043]
 [0.83630435 0.83630435 0.82891304 0.82847826 0.83586957 0.83543478
  0.8226087 ]
 [0.835      0.835      0.8123913  0.81195652 0.83478261 0.83456522
  0.82978261]
 [0.83586957 0.83586957 0.83652174 0.83586957 0.83543478 0.83521739
  0.82434783]
 [0.83869565 0.83869565 0.83891304 0.83847826 0.83847826 0.83869565
  0.8226087 ]
 [0.8319565

C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [77]:
np.mean(accuracies, axis = 0) 

array([0.83502174, 0.83502174, 0.83124783, 0.8310087 , 0.83515217,
       0.83518261, 0.82258696])

In [78]:
np.std(accuracies, axis = 0)

array([0.00426514, 0.00426514, 0.00919304, 0.00928378, 0.00429319,
       0.00428506, 0.00412454])

In [79]:
np.mean(auc_accuracies, axis = 0)

array([0.89384806, 0.89522376, 0.51016482, 0.51009598, 0.90013188,
       0.9001223 , 0.52419603])

In [80]:
np.std(auc_accuracies, axis = 0)

array([0.00701683, 0.00708372, 0.0144523 , 0.01449405, 0.00710436,
       0.00706523, 0.0105884 ])

# MNIST

In [81]:
from sklearn.datasets import fetch_openml

In [82]:
mnist = fetch_openml('mnist_784')

# Extract features (pixel values) and target labels
X = mnist.data.astype('float32')
y = mnist.target.astype('int64')


label_counts = np.bincount(y)

digit_filter = (y == 5) | (y == 8)
X = X[digit_filter]
y = y[digit_filter]

np.unique(y)

n5 = np.count_nonzero(y == 5)
n8 = np.count_nonzero(y == 8)

y = y.replace({5 : 1, 8 : 0})
y.value_counts()

C:\Users\panda\anaconda3\lib\site-packages\sklearn\datasets\_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


0    6825
1    6313
Name: class, dtype: int64

In [83]:
# Number of times to repeat the process
num_repeats = 50

# Initialize an empty matrix (10-by-4) to store accuracies
accuracies = np.zeros((num_repeats, 7))
auc_accuracies = np.zeros((num_repeats, 7))

seed = 42
# Repeat the process and store accuracies
for i in range(num_repeats):
    np.random.seed(seed)
    accuracies[i], auc_accuracies[i] = iterate_process(X, y)
    seed += 2
    

# Print the accuracies
print("Accuracies:", accuracies)
print("AUC:", auc_accuracies)

C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

Accuracies: [[0.96689498 0.96689498 0.96270928 0.96251903 0.96746575 0.96727549
  0.94501522]
 [0.96289954 0.96289954 0.95871385 0.95871385 0.96289954 0.9630898
  0.94824962]
 [0.96023592 0.96023592 0.95585997 0.95605023 0.96004566 0.96023592
  0.93436073]
 [0.9608067  0.9608067  0.95928463 0.95890411 0.96156773 0.96175799
  0.92979452]
 [0.96175799 0.96175799 0.95966514 0.95966514 0.96137747 0.96137747
  0.94292237]
 [0.95947489 0.95947489 0.95319635 0.95319635 0.95928463 0.95928463
  0.9326484 ]
 [0.96156773 0.96156773 0.95890411 0.95890411 0.96156773 0.96156773
  0.94101979]
 [0.96213851 0.96213851 0.96023592 0.96004566 0.96175799 0.96213851
  0.9423516 ]
 [0.96860731 0.96860731 0.96385084 0.96366058 0.96841705 0.96860731
  0.94577626]
 [0.96423135 0.96423135 0.96423135 0.96423135 0.96480213 0.96480213
  0.94197108]
 [0.96499239 0.96499239 0.96289954 0.96270928 0.96556317 0.96594368
  0.94349315]
 [0.96461187 0.96461187 0.95871385 0.95890411 0.96480213 0.96499239
  0.94311263]
 [0.9

C:\Users\panda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [84]:
np.mean(accuracies, axis = 0) 

array([0.96216514, 0.96216514, 0.95961568, 0.95960426, 0.96230974,
       0.96232877, 0.94001903])

In [85]:
np.std(accuracies, axis = 0)

array([0.00271691, 0.00271691, 0.00272909, 0.00269508, 0.00277027,
       0.00278663, 0.00500938])

In [86]:
np.mean(auc_accuracies, axis = 0)

array([0.99183995, 0.99186007, 0.98576107, 0.98575708, 0.99325966,
       0.99325203, 0.97668592])

In [87]:
np.std(auc_accuracies, axis = 0)

array([0.00095164, 0.00095131, 0.00292342, 0.00292118, 0.00074406,
       0.00074722, 0.0029753 ])